Ref: https://www.youtube.com/watch?v=3ZacJ9zRVOU 
01. Data Collection
02. Data Preprocessing
03. EDA
04. Conclusions

In [11]:
from bs4 import BeautifulSoup as soup
from datetime import date, datetime
from urllib.request import Request, urlopen

import pandas as pd
import re

# Web Scrapping

In [9]:
today = datetime.now()
today
yesterday_str = "%s %d,%d" %(date.today().strftime("%b"), today.day-1, today.year)
yesterday_str

'Jun 19,2021'

In [3]:
url = 'https://www.worldometers.info/coronavirus/#countries'
req = Request(url, headers={'User-Agent':"Morilla/5.0"})

webpage = urlopen(req)
page_soup = soup(webpage, 'html.parser')
# page_soup

In [4]:
table = page_soup.findAll("table",{"id":"main_table_countries_yesterday"})
table

[<table class="table table-bordered table-hover main_table_countries" id="main_table_countries_yesterday" style="width:100%;margin-top: 0px !important;display:none;">
 <thead>
 <tr>
 <th width="1%">#</th>
 <th width="100">Country,<br/>Other</th>
 <th width="20">Total<br/>Cases</th>
 <th width="30">New<br/>Cases</th>
 <th width="30">Total<br/>Deaths</th>
 <th width="30">New<br/>Deaths</th>
 <th width="30">Total<br/>Recovered</th>
 <th width="30">New<br/>Recovered</th>
 <th width="30">Active<br/>Cases</th>
 <th width="30">Serious,<br/>Critical</th>
 <th width="30">Tot Cases/<br/>1M pop</th>
 <th width="30">Deaths/<br/>1M pop</th>
 <th width="30">Total<br/>Tests</th>
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>
 <th width="30">Population</th>
 <th style="display:none" width="30">Continent</th>
 <th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
 <th width="30">New Cases/1M pop</th>
 <th width="30">New 

### Handle labels

In [77]:
containers = table[0].findAll("tr", {"style":""})

col_labels = []

label_data = containers.pop(0)
label_data = label_data.findAll("th",{})

for label in label_data:
    col_labels.append(label.text.replace("\n", ""))
    
col_labels.pop(0) # remove 
col_labels

['Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'NewRecovered',
 'ActiveCases',
 'Serious,Critical',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/1M pop',
 'Population',
 'Continent',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl',
 'New Cases/1M pop',
 'New Deaths/1M pop',
 'Active Cases/1M pop']

In [24]:
data = []

for country in containers:
    country_data = []
    country_container = country.findAll("td")
    
    for i in range(1, len(country_container)):
        feature = country_container[i].text
        
        feature = re.sub('[^A-Za-z0-9]+', '', feature)
        if feature == '': 
            feature = '-1'
        country_data.append(feature)
        
    data.append(country_data)

In [25]:
df = pd.DataFrame(data)
df 

0          1       2        3     4          5       6   \
0                  World  178950030  359757  3875334  8179  163471930  364639   
1                  China      91564      30     4636    -1      86425      28   
2                    USA   34401712    7782   617083   170   28694843   18914   
3                  India   29881352   58588   386740  1239   28758447   87568   
4                 Brazil   17883750   81574   500868  2247   16183849   46881   
..                   ...        ...     ...      ...   ...        ...     ...   
203                Macao         53      -1       -1    -1         51      -1   
204            Greenland         49      -1       -1    -1         40      -1   
205  SaintPierreMiquelon         26      -1       -1    -1         25      -1   
206        WesternSahara         10      -1        1    -1          8      -1   
207                Total  178950030  359757  3875334  8179  163471930  364639   

           7      8       9   ...         11       12          13  \
0    11602766  82769   22958  ...         -1       -1          -1   
1         503     21      64  ...  160000000   111163  1439323776   
2     5089786   3989  103347  ...  498689211  1498123   332876117   
3      736165   8944   21450  ...  389207637   279385  1393086773   
4     1199033   8318   83562  ...   52714701   246310   214017366   
..        ...    ...     ...  ...        ...      ...         ...   
203         2     -1      81  ...       4483     6815      657861   
204         9     -1     862  ...      27387   481614       56865   
205         1     -1    4508  ...       6128  1062598        5767   
206         1     -1      16  ...         -1       -1      611440   
207  11602766  82769  229576  ...         -1       -1          -1   

               14     15      16   17   18  19     20  
0             All     -1      -1   -1   -1  -1     -1  
1            Asia  15719  310467    9  002  -1     03  
2    NorthAmerica     10     539    1   23  05  15290  
3            Asia     47    3602    4   42  09    528  
4    SouthAmerica     12     427    4  381  10   5603  
..            ...    ...     ...  ...  ...  ..    ...  
203          Asia  12412      -1  147   -1  -1      3  
204  NorthAmerica   1161      -1    2   -1  -1    158  
205  NorthAmerica    222      -1    1   -1  -1    173  
206        Africa  61144  611440   -1   -1  -1      2  
207           All     -1      -1   -1   -1  -1     -1  

[208 rows x 21 columns]

In [81]:
df.columns = col_labels
df.head()

Country,Other TotalCases NewCases TotalDeaths NewDeaths TotalRecovered  \
0         World  178950030   359757     3875334      8179      163471930   
1         China      91564       30        4636        -1          86425   
2           USA   34401712     7782      617083       170       28694843   
3         India   29881352    58588      386740      1239       28758447   
4        Brazil   17883750    81574      500868      2247       16183849   

  NewRecovered ActiveCases Serious,Critical Tot Cases/1M pop  ... TotalTests  \
0       364639    11602766            82769            22958  ...         -1   
1           28         503               21               64  ...  160000000   
2        18914     5089786             3989           103347  ...  498689211   
3        87568      736165             8944            21450  ...  389207637   
4        46881     1199033             8318            83562  ...   52714701   

  Tests/1M pop  Population     Continent 1 Caseevery X ppl 1 Deathevery X ppl  \
0           -1          -1           All                -1                 -1   
1       111163  1439323776          Asia             15719             310467   
2      1498123   332876117  NorthAmerica                10                539   
3       279385  1393086773          Asia                47               3602   
4       246310   214017366  SouthAmerica                12                427   

  1 Testevery X ppl New Cases/1M pop New Deaths/1M pop Active Cases/1M pop  
0                -1               -1                -1                  -1  
1                 9              002                -1                  03  
2                 1               23                05               15290  
3                 4               42                09                 528  
4                 4              381                10                5603  

[5 rows x 21 columns]